In [76]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

In [77]:
dossier="/Users/gilles/Github/eCloze/"
enonce="MCLangueOrale2ePart5"
theme="MCLangueOrale2ePart5"

In [78]:
from ToolsMoodleXML2 import *
import ToolsMoodleXML
import yaml

In [79]:
yamlFichier=dossier+enonce+".yaml"
csvFichier=dossier+theme+".txt"
boolEX=False
titreNoEX=False

In [80]:
with codecs.open(yamlFichier, 'r',encoding="utf8") as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r',encoding="utf8") as file:
    donneesLignes=file.readlines()
    

In [81]:
def donneesEX():
    titre=[]
    result=""
    i=0
    for ligne in donneesLignes:
        i+=1
        if ligne.startswith("#EX;"):
            if result!="":
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            result["boucle"].append(temp)
            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            donnees.append(ligne)
    return

def donneesNoLine():
    donnees.append("")
    for ligne in donneesLignes:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            donnees[-1]+=ligne+"<br>"
        elif ligne.strip().strip(";")=="":
            donnees.append("")

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
#    print "avec #EX;"
    donneesEX()
elif not boolEX:
    donneesNoLine()
else:
#    print "sans #EX;"
    donneesNoEX()

In [82]:
if u"Pénalité" in configuration:
    ToolsMoodleXML.penalite=configuration[u"Pénalité"]
if u"NbChoix" in configuration:
    ToolsMoodleXML.maxChoix=configuration["NbChoix"]-1 #maxChoix alternatives à la solution pour avoir NbChoix en tout

In [83]:
categorie=configuration[u"Catégorie"]
titreGeneral=configuration[u"Titre"]
questions=XMLSeries(categorie)
debutConsigne=configuration[u"Consigne"][u"début"]
finConsigne=configuration[u"Consigne"][u"fin"]

In [84]:
def escapeDieseTilde(chaine):
    result=chaine
    m=re.search(ur"(?<!\\)(#|~)",result)
    while m:
        print m.group(0),m.group(1)   
        result=result.replace(m.group(0),ur"\%s"%m.group(1))
        m=re.search(ur"(?<!\\)(#|~)",result)
    return result

def questionParentheses(chaine):
    result=chaine
#    m=re.search(ur"\(([^)]+)\)",result)
    m=re.search(ur"(?<!\\)\((.+)(?<!\\)\)",result)
    while m:
        result=result.replace(m.group(0),u"{1:SAC:=%s}"%m.group(1))
        m=re.search(ur"(?<!\\)\((.+)(?<!\\)\)",result)
    result=result.replace(ur"\(","(").replace(ur"\)",")")
    return result

def questionCrochets(chaine):
    result=chaine
    m=re.search(ur"(?<!\\)\[(([^\]]|\\\])+)(?<!\\)\]",result)
    while m:
        resultMC="~".join(m.group(1).split(";"))
        print resultMC
        result=result.replace(m.group(0),u"{1:MC:=%s}"%resultMC)
        m=re.search(ur"(?<!\\)\[(([^\]]|\\\])+)(?<!\\)\]",result)
    result=result.replace(ur"\[","[").replace(ur"\]","]")
    return result

def questionParCro(chaine):
    chaineEsc=escapeDieseTilde(chaine)
    chainePar=questionParentheses(chaineEsc)
    chaineCro=questionCrochets(chainePar)
    return chaineCro

In [85]:
print questionParCro(ur"Ceci est [une;un;des] réponse \((courte)\), cela est un [choix multiple;\[schmilblick\];truc~bizarre;bazar;#EX].")

~ ~
# #
une~un~des
choix multiple~\[schmilblick\]~truc\~bizarre~bazar~\#EX
Ceci est {1:MC:=une~un~des} réponse ({1:SAC:=courte}), cela est un {1:MC:=choix multiple~[schmilblick]~truc\~bizarre~bazar~\#EX}.


In [86]:
for numEx,exercices in enumerate(donnees):
    titre="%s%02d"%(titreGeneral, numEx+1)
    corps=debutConsigne+questionParCro(exercices)+finConsigne
    print corps
    questions.addExercice(ClozeExercice(titre,corps))

계절을~게저를~게절을~개저를
꽃처럼~꼬처럼~꼬초럼~꽃초롬
생각에는~생가게는~샌가게는~샌가개는
같아요~가타요~갖타요~갓타요
<br/>﻿이해 Compréhension orale 5
<br>맞는 철자를 고르세요. Ecoutez et choisissez l’orthographe correcte.<br>
<br>17.{1:MC:=계절을~게저를~게절을~개저를}<br>
<br>18.{1:MC:=꽃처럼~꼬처럼~꼬초럼~꽃초롬}<br>
<br>19.{1:MC:=생각에는~생가게는~샌가게는~샌가개는}<br>
<br>20.{1:MC:=같아요~가타요~갖타요~갓타요}<br>
<br><br/>


In [87]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(questions.getSeries())

In [88]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [89]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())

In [90]:
print questionParCro(donnees[0])

계절을~게저를~게절을~개저를
꽃처럼~꼬처럼~꼬초럼~꽃초롬
생각에는~생가게는~샌가게는~샌가개는
같아요~가타요~갖타요~갓타요
﻿이해 Compréhension orale 5
<br>맞는 철자를 고르세요. Ecoutez et choisissez l’orthographe correcte.<br>
<br>17.{1:MC:=계절을~게저를~게절을~개저를}<br>
<br>18.{1:MC:=꽃처럼~꼬처럼~꼬초럼~꽃초롬}<br>
<br>19.{1:MC:=생각에는~생가게는~샌가게는~샌가개는}<br>
<br>20.{1:MC:=같아요~가타요~갖타요~갓타요}<br>
<br>
